# Векторный поиск

In [ ]:
import numpy as np
import pandas as pd
import pymorphy2
import re
import nltk
from nltk.corpus import stopwords
from scipy import spatial
import json
import string

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# откроем инвертированный список лемм (создан в 4 задании)
with open('inverted_index_lemmas.txt', encoding="utf-8") as file:
    rows = [row.strip() for row in file]

index_lemmas = {}
lemmas = list()
for row in rows:
    row = row.replace('\'', '\"')
    dict_ = json.loads(row)
    lemmas.append(dict_["word"])
    index_lemmas[dict_["word"]] = [dict_["count"], dict_["inverted_array"]]

In [ ]:
# обработка контента (токенизация, удаление стоп-слов, знаков препинания)
def tokenize(content):
    tokens_ = nltk.word_tokenize(content)
    tokens_ = [i.lower() for i in tokens_]
    tokens_ = [i for i in tokens_ if ( i not in string.punctuation )]
    tokens_ = filter(None, [re.sub(r"[^a-zа-я-]+", r"", i) for i in tokens_])

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'так', 'вот', 'как', 'в', 'к',
                         'на', 'о', 'при', 'из-за', 'за', 'ао', 'но', 'х',
                         'хотя', 'среди', 'помимо', 'с'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words )]

    stop_words2 = stopwords.words('english')
    stop_words2.extend(['the', 'e', 'a', 'd', 'b', 'x', 'c'])
    tokens_ = [i for i in tokens_ if ( i not in stop_words2 )]

    tokens_ = [i.replace("«", "").replace("»", "") for i in tokens_]

    return tokens_

In [ ]:
# создание матрицы по индексу лемм
# строки - документы, столбцы - леммы, пересечения - существует ли данная лемма в данном документе
def get_matrix_lemmas(index_lemmas):
  id = 0
  matrix = []
  for i in range(len(index_lemmas)):
    matrix.append([0] * 100)
  for k in index_lemmas:
    docs = index_lemmas[k][1]
    for doc in docs:
        matrix[id][int(doc) - 1] = 1
    id += 1
  return np.array(matrix).transpose()

In [ ]:
get_matrix_lemmas(index_lemmas).shape

(100, 4365)

In [ ]:
# создание вектора размером кол-ва лемм, значения которого 0 или 1 в зависимости от лемм поисковой фразы
def get_vector_from_search_query(search):
    morph = pymorphy2.MorphAnalyzer()
    tokens = tokenize(search)
    lemmas_from_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    vector = [0] * len(lemmas)
    for token in lemmas_from_tokens:
        if token in lemmas:
            vector[lemmas.index(token)] = 1
    return vector

In [ ]:
matrix = get_matrix_lemmas(index_lemmas)

In [ ]:
# Функция поиска: строки матрицы сравниваем с вектором через косинусное расстояние, 
# и записываем для каждого документа значение - степень сходства
# чем ближе значение косинуса к 1, тем ближе угол к 0 градусам, то есть тем более похожи два вектора
def search(search):
    vector = get_vector_from_search_query(search)
    docs = dict()
    for id, doc in enumerate(matrix):
        docs[id + 1] = spatial.distance.cosine(vector, doc)
    sorted_ = sorted(docs.items(), key=lambda x: x[1])
    sorted_docs = []
    for item in sorted_:
        sorted_docs.append(item[0])
    return sorted_docs

In [ ]:
def search_top5(search_):
  docs = search(search_)
  print(docs[:5])

In [ ]:
result_docs = search('Тело упадет со скоростью')
result_docs[:20]

[87, 38, 7, 62, 16, 98, 95, 69, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13]

In [ ]:
search_top5('Тело упадет со скоростью')

[87, 38, 7, 62, 16]
